In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-10-07 17:31:44 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-10-07 17:31:47 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=7, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-10-07 17:31:47 __main__ INFO     transformers.__version__='4.55.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
# model_key = "meta-llama/Llama-3.1-70B-Instruct"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

2025-10-07 17:31:50 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-10-07 17:31:50 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-10-07 17:31:50 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-10-07 17:31:50 wandb.docker.auth DEBUG    No config file found


## Load LM and the Select Task

In [4]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # torch_dtype=torch.float32,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
    attn_implementation="eager",
)

2025-10-07 17:31:51 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-10-07 17:31:51 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-10-07 17:31:51 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-10-07 17:31:51 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-10-07 17:31:51 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/meta-llama/Llama-3.3-70B-Instruct/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-10-07 17:32:33 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-10-07 17:32:33 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0
2025-10-07 17:32:34 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [5]:
from src.selection.data import SelectOneTask

#################################################################################
# TASK_CLS = SelectOrderTask
# prompt_template_idx = 1
TASK_CLS = SelectOneTask
prompt_template_idx = 3
N_DISTRACTORS = 5
OPTION_STYLE = "single_line"
#################################################################################

select_task = TASK_CLS.load(
    path=os.path.join(
        env_utils.DEFAULT_DATA_DIR, 
        "selection", 
        # "profession.json"
        # "nationality.json"
        "objects.json"
    )
)

print(select_task)

['name', 'prompt_templates', 'odd_one_prompt_templates', 'order_prompt_templates', 'count_prompt_templates', 'yes_no_prompt_templates', 'first_item_in_cat_prompt_templates', 'last_item_in_cat_prompt_templates', 'categories', 'exclude_categories']
SelectOneTask: (different objects)
Categories: fruit(15), vehicle(15), furniture(15), animal(15), music instrument(15), clothing(15), electronics(14), sport equipment(15), kitchen appliance(15), vegetable(14), building(15), office supply(15), bathroom item(15), flower(15), tree(15), jewelry(15)



In [6]:
import copy
import random
from src.selection.utils import KeyedSet, get_first_token_id, verify_correct_option
from src.selection.data import SelectionSample
from src.functional import predict_next_token
from src.tokens import prepare_input
from src.selection.data import get_counterfactual_samples_within_task

In [7]:
patch_sample, clean_sample = get_counterfactual_samples_within_task(
    # patch_category="politician",
    # clean_category="actor",
    mt=mt,
    task=select_task,
    patch_category="fruit",
    clean_category="vehicle",
    filter_by_lm_prediction=True,
    prompt_template_idx=prompt_template_idx,
    option_style=OPTION_STYLE,
    distinct_options=True,
    patch_n_distractors=5,
    clean_n_distractors=5
)

# patch_sample.default_option_style = "single_line"
# clean_sample.default_option_style = "numbered"

print(patch_sample.prompt(), ">>", patch_sample.obj)
print(clean_sample.prompt(), ">>", clean_sample.obj)

2025-10-07 17:32:34 src.selection.data INFO     clean_obj_idx=4 | ['Lotion', 'Zebra', 'Mango', 'Table', 'Boat', 'Baseball']
type(task)=<class 'src.selection.data.SelectOneTask'>
2025-10-07 17:32:35 src.selection.data INFO     Options: Theater, Grape, Violin, Tiger, Cabinet, Bike.
Which among these objects mentioned above is a fruit?
Answer:
2025-10-07 17:32:35 src.selection.data INFO     Peach | fruit -> Grape | pred=['" Grape"[80629] (p=0.738, logit=21.000)', '" The"[578] (p=0.088, logit=18.875)', '" Among"[22395] (p=0.061, logit=18.500)', '" A"[362] (p=0.047, logit=18.250)', '" GRA"[65120] (p=0.007, logit=16.375)']
2025-10-07 17:32:35 src.selection.data INFO     Options: Lotion, Zebra, Mango, Table, Boat, Baseball.
Which among these objects mentioned above is a vehicle?
Answer:
2025-10-07 17:32:35 src.selection.data INFO     Bus | vehicle -> Boat | pred=['" Boat"[45332] (p=0.727, logit=22.000)', '" A"[362] (p=0.111, logit=20.125)', '" The"[578] (p=0.077, logit=19.750)', '" Among"[223

## Training Set

In [8]:
# from src.selection.data import CounterFactualSamplePair
# from src.functional import free_gpu_cache
# from src.selection.data import get_counterfactual_samples_interface
# import random

# train_samples_save_path = os.path.join(
#     env_utils.DEFAULT_RESULTS_DIR,
#     "selection",
#     "samples",
#     "train",
#     mt.name.split("/")[-1],
#     select_task.task_name,
#     "objects",
#     # "profession",
#     # "nationality",
#     # "landmarks"
#     # "rhymes"
# )

# os.makedirs(train_samples_save_path, exist_ok=True)


# free_gpu_cache()
# train_set = []
# train_limit = 1024
# start_from = 1

# counterfactual_sampler = get_counterfactual_samples_interface[select_task.task_name]

# while len(train_set) < train_limit:
#     print(f"sample {len(train_set)+1} / {train_limit}")
#     patch, clean = counterfactual_sampler(
#         mt=mt,
#         task=select_task,
#         filter_by_lm_prediction=True,
#         prompt_template_idx=3,
#         option_style=OPTION_STYLE,
#         n_distractors=random.choice(range(2, 6)),
#     )
#     train_set.append((clean, patch))
#     cf_pair = CounterFactualSamplePair(
#         patch_sample=patch,
#         clean_sample=clean,
#     )
#     cf_pair.detensorize()
#     with open(
#         os.path.join(
#             train_samples_save_path,
#             f"{len(train_set) + start_from - 1:05d}.json",
#         ),
#         "w",
#     ) as f:
#         json.dump(cf_pair.to_dict(), f, indent=2)

# len(train_set)

In [9]:
from src.selection.data import CounterFactualSamplePair
import random

train_set = []
train_limit = 1024

train_samples_load_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "selection",
    "samples",
    "train",
    mt.name.split("/")[-1],
    select_task.task_name,
    "objects",
    # "profession",
    # "nationality"
    # "landmarks",
    # "rhymes",
)

sample_files = [
    os.path.join(train_samples_load_path, f)
    for f in os.listdir(train_samples_load_path)
    if f.endswith(".json")
]
logger.info(f"Found {len(sample_files)} sample files")

prefix = ""
# prefix = "Recall the nationality of these people:\n"
# prefix = "Recall which country these landmarks are located in:\n"
# prefix = "Think about how these words sound when you say them aloud:\n"

random.shuffle(sample_files)
sample_files = sample_files[:train_limit]
for sample_file in sample_files:
    with open(sample_file, "r") as f:
        cf_pair_data = json.load(f)
    cf_pair = CounterFactualSamplePair.from_dict(cf_pair_data)
    # cf_pair.patch_sample.default_option_style = "bulleted"
    # cf_pair.clean_sample.default_option_style = "bulleted"

    # patch_category = cf_pair.patch_sample.category
    # random_category = random.choice(list(set(select_task.categories) - {patch_category}))
    # random_obj = random.choice(select_task.category_wise_examples[random_category])
    # cf_pair.patch_sample.options[cf_pair.patch_sample.obj_idx] = random_obj

    cf_pair.clean_sample.prompt_template = prefix + cf_pair.clean_sample.prompt_template
    cf_pair.patch_sample.prompt_template = prefix + cf_pair.patch_sample.prompt_template
    train_set.append((cf_pair.clean_sample, cf_pair.patch_sample))

len(train_set)

2025-10-07 17:32:36 __main__ INFO     Found 1024 sample files


1024

In [10]:
destination, source = random.choice(train_set)
print("Destination:", destination.prompt(), ">>", destination.obj)
print("Source:", source.prompt(), ">>", source.obj)

Destination: Options: Lily, Pen, Socks, Harp, Air fryer, Table.
Which among these objects mentioned above is a music instrument?
Answer: >> Harp
Source: Options: Chain, Paper, Food processor, Trumpet, Keyboard, Carnation.
Which among these objects mentioned above is a flower?
Answer: >> Carnation


## Validation Set

In [11]:
# from src.selection.data import CounterFactualSamplePair
# from src.functional import free_gpu_cache
# from src.selection.data import get_counterfactual_samples_interface
# import random

# data_type = "objects"
# # data_type = "profession"
# # data_type = "nationality"
# select_val_task = TASK_CLS.load(
#     path=os.path.join(
#         env_utils.DEFAULT_DATA_DIR, 
#         "selection", 
#         f"{data_type}.json"
#     )
# )

# validation_samples_save_path = os.path.join(
#     env_utils.DEFAULT_RESULTS_DIR,
#     "selection",
#     "samples",
#     "validation",
#     mt.name.split("/")[-1],
#     select_val_task.task_name,
#     data_type,
# )

# os.makedirs(validation_samples_save_path, exist_ok=True)

    
# free_gpu_cache()
# validation_set = []
# validation_limit = 512
# start_from = 0

# counterfactual_sampler = get_counterfactual_samples_interface[select_task.task_name]

# while len(validation_set) < validation_limit:
#     print(f"sample {len(validation_set)+1} / {validation_limit}")
#     patch, clean = counterfactual_sampler(
#         mt=mt,
#         task=select_val_task,
#         filter_by_lm_prediction=True,
#         prompt_template_idx=3,
#         option_style=OPTION_STYLE,
#         n_distractors=random.choice(range(2, 6)),
#     )
#     validation_set.append((clean, patch))
#     cf_pair = CounterFactualSamplePair(
#         patch_sample=patch,
#         clean_sample=clean,
#     )
#     cf_pair.detensorize()
#     with open(
#         os.path.join(
#             validation_samples_save_path,
#             f"{len(validation_set) + start_from - 1:05d}.json",
#         ),
#         "w",
#     ) as f:
#         json.dump(cf_pair.to_dict(), f, indent=2)

# len(validation_set)

In [12]:
from src.selection.data import CounterFactualSamplePair
import random

# data_type = "profession"
data_type = "objects"

validation_set = []
validation_limit = 512

validation_samples_load_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "selection",
    "samples",
    "validation",
    mt.name.split("/")[-1],
    select_task.task_name,
    data_type,
)

sample_files = [
    os.path.join(validation_samples_load_path, f)
    for f in os.listdir(validation_samples_load_path)
    if f.endswith(".json")
]
logger.info(f"Found {len(sample_files)} sample files")

prefix = ""
# prefix = "Recall the nationality of these people:\n"
# prefix = "Recall which country these landmarks are located in:\n"
# prefix = "Think about how these words sound when you say them aloud:\n"

random.shuffle(sample_files)
sample_files = sample_files[:validation_limit]
for sample_file in sample_files:
    with open(sample_file, "r") as f:
        cf_pair_data = json.load(f)
    cf_pair = CounterFactualSamplePair.from_dict(cf_pair_data)
    # cf_pair.patch_sample.default_option_style = "bulleted"
    # cf_pair.clean_sample.default_option_style = "bulleted"

    cf_pair.clean_sample.prompt_template = prefix + cf_pair.clean_sample.prompt_template
    cf_pair.patch_sample.prompt_template = prefix + cf_pair.patch_sample.prompt_template
    validation_set.append((cf_pair.clean_sample, cf_pair.patch_sample))

len(validation_set)

2025-10-07 17:32:38 __main__ INFO     Found 1024 sample files


512

## Debug SVD patching logic

In [13]:
clean_sample, patch_sample = train_set[17]
print(clean_sample.prompt(), ">>", clean_sample.obj)
print(patch_sample.prompt(), ">>", patch_sample.obj)

Options: Shower, Tulip, Surfboard, Dolphin, Cabinet.
Which among these objects mentioned above is a animal?
Answer: >> Dolphin
Options: Tie, Anklet, Headphones, Giraffe, Marigold.
Which among these objects mentioned above is a flower?
Answer: >> Marigold


In [14]:
import baukit
from src.hooking.llama_attention import LlamaAttentionPatcher
import types
from src.functional import get_module_nnsight

###################################################################################
batch_size = 1  # tokenized.input_ids.shape[0]
n_heads = mt.config.num_attention_heads
head_dim = get_module_nnsight(mt._model, mt.attn_module_name_format.format(0)).head_dim
query_idx = -1  # almost always the last token
###################################################################################

mt.reset_forward()
mt.set_attn_implementation("sdpa")

layer_idx, head_idx = (35, 19)

attn_block_name = mt.attn_module_name_format.format(layer_idx)
attn_block = baukit.get_module(mt._model, attn_block_name)
attn_block.forward = types.MethodType(
    LlamaAttentionPatcher(block_name=attn_block_name),
    attn_block,
)

patch_tokenized = prepare_input(prompts=patch_sample.prompt(), tokenizer=mt)
patch_seq_len = patch_tokenized.input_ids.shape[1]
input_ln = mt.layer_name_format.format(layer_idx) + ".input_layernorm"

with mt.trace(patch_tokenized) as trace:
    ln_module = get_module_nnsight(mt, input_ln)
    patch_ln = ln_module.output.save()

    q_proj_name = mt.attn_module_name_format.format(layer_idx) + ".q_proj"
    q_proj_module = get_module_nnsight(mt, q_proj_name)
    print(f"{q_proj_module.output.shape=}")
    patch_q_proj = (
        q_proj_module.output.view(batch_size, patch_seq_len, n_heads, head_dim)
        .transpose(1, 2)
        .save()
    )
    # patch_q_proj = PatchSpec(
    #     location=(q_proj_name + f".{head_idx}", -1),
    #     patch=patch_q_proj[:, head_idx, query_idx, :].squeeze().save()
    # )

q_proj_module.output.shape=torch.Size([1, 30, 8192])


In [15]:
head_q_out = patch_q_proj[:, head_idx, query_idx, :]
head_inp = patch_ln[:, query_idx, :]

head_q_out.shape, head_inp.shape

(torch.Size([1, 128]), torch.Size([1, 8192]))

In [16]:
attn_block = baukit.get_module(mt._model, attn_block_name)
q_proj_per_head = attn_block.q_proj.weight.T.view(attn_block.q_proj.out_features, n_heads, head_dim)
q_proj_per_head.shape

torch.Size([8192, 64, 128])

In [17]:
q_proj_head = q_proj_per_head[:, head_idx, :]
q_proj_head.shape

torch.Size([8192, 128])

In [18]:
torch.allclose(
    head_q_out,
    torch.matmul(head_inp, q_proj_head),
    atol=1e-3
)

True

In [19]:
from src.utils.typing import SVD
svd = SVD.calculate(q_proj_head)
svd.U.shape, svd.S.shape, svd.V.shape

(torch.Size([8192, 128]), torch.Size([128]), torch.Size([128, 128]))

In [20]:
masks = torch.zeros_like(svd.S)
masks[:5] = 1.0
U_k = svd.U * masks[None, :]        # 8192 × 128
S_k = svd.S * masks                 # 128
Vh_k = svd.V.T * masks[:, None]     # 128 × 128

# Reconstruct the rank-5 approximation of q_proj_head
q_proj_head_rank5 = U_k @ torch.diag(S_k) @ Vh_k  # 8192 × 128

# Project the input through the rank-5 transformation
q_out_proj_1 = head_inp @ q_proj_head_rank5  # 1 × 128

In [21]:
torch.allclose(
    q_out_proj_1,
    head_q_out @ Vh_k.T @ Vh_k,
    atol=1e-2
)

True

### Saving the SVD of the query projection matrix per head

In [22]:
from tqdm.auto import tqdm
import baukit

head_svd_save_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "q_proj_svd",
    mt.name.split("/")[-1],
)

os.makedirs(head_svd_save_path, exist_ok=True)
for layer_idx in tqdm(range(mt.n_layer)):
    attn_block_name = mt.attn_module_name_format.format(layer_idx)
    attn_block = baukit.get_module(mt._model, attn_block_name)
    q_proj_per_head = attn_block.q_proj.weight.T.view(attn_block.q_proj.out_features, n_heads, head_dim)
    for head_idx in range(n_heads):
        q_proj_head = q_proj_per_head[:, head_idx, :]
        svd = SVD.calculate(q_proj_head)
        with open(os.path.join(head_svd_save_path, f"{layer_idx}_{head_idx}.pt"), "wb") as f:
            torch.save(svd, f)

  0%|          | 0/80 [00:00<?, ?it/s]

### Loading the SVD of the query projection matrix per head

In [23]:
from tqdm.auto import tqdm
from src.functional import free_gpu_cache, get_module_nnsight
from src.utils.typing import SVD

head_svd_load_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "q_proj_svd",
    mt.name.split("/")[-1],
)

q_proj_basis_directions = {}
head_dim = get_module_nnsight(mt._model, mt.attn_module_name_format.format(0)).head_dim
n_embd = mt.n_embd
n_heads = mt.config.num_attention_heads

for layer_idx in tqdm(range(mt.n_layer)):
    for head_idx in range(n_heads):
        with open(os.path.join(head_svd_load_path, f"{layer_idx}_{head_idx}.pt"), "rb") as f:
            svd = torch.load(f, weights_only=False)
        assert isinstance(svd, SVD)
        assert svd.U.shape == (n_embd, head_dim)
        assert svd.S.shape == (head_dim,)
        assert svd.V.shape == (head_dim, head_dim)

        q_proj_basis_directions[(layer_idx, head_idx)] = svd.V.T #! the transpose here is important!

  0%|          | 0/80 [00:00<?, ?it/s]

In [24]:
len(q_proj_basis_directions)

5120

In [25]:
q_proj_basis_directions[(35, 19)].shape

torch.Size([128, 128])

## optimization logic

In [26]:
from src.utils.typing import PathLike
from torch.optim import AdamW
from src.functional import free_gpu_cache
from src.tokens import TokenizerOutput
from typing import Literal
from src.selection.optimization import promote_target_suppress_distractors, match_gold_logit_distribution, cache_q_projections_prev
import numpy as np

def apply_q_proj_patch_with_projection(
    mt: ModelandTokenizer,
    source_tokenized: TokenizerOutput,
    destination_tokenized: TokenizerOutput,
    projections: dict[tuple[int, int], torch.Tensor],
    token_indices: list[int],
):
    q_proj_modules = []
    layer_to_heads = {}
    query_locations = []
    for layer_idx, head_idx in projections.keys():
        module_name = mt.attn_module_name_format.format(layer_idx) + ".q_proj"
        q_proj_modules.append(module_name)
        if layer_idx not in layer_to_heads:
            layer_to_heads[layer_idx] = []
        layer_to_heads[layer_idx].append(head_idx)
        query_locations.extend(
            (layer_idx, head_idx, query_idx)
            for query_idx in token_indices
        )

    q_projections = cache_q_projections_prev(
        mt=mt,
        input=source_tokenized,
        query_locations=query_locations,
        return_output=False,
    )
    patches = {}
    for (layer_idx, head_idx, query_idx), q_proj in q_projections.items():
        module_name = (
            mt.attn_module_name_format.format(layer_idx) + ".q_proj"
        )
        patches[(module_name, head_idx)] = (layer_idx, q_proj)
    
    patch_q_states = patches
    batch_size = destination_tokenized.input_ids.shape[0]
    seq_len = destination_tokenized.input_ids.shape[1]

    def perform_patch(repr, layer_name):
        if layer_name not in q_proj_modules:
            return repr
        # logger.debug(f"Patching at layer: {layer_name}")
        layer_idx = int(layer_name.split(".")[2])
        repr = repr.view(batch_size, seq_len, n_heads, head_dim).transpose(1, 2)
        for head_idx in layer_to_heads[layer_idx]:
            if (layer_idx, head_idx) not in projections:
                assert False, f"{(layer_idx, head_idx)} not in projections. This should never happen!"
            projection = projections[(layer_idx, head_idx)]
            q_clean = repr[:, head_idx, token_indices, :]
            layer_idx, q_patch = patch_q_states[(layer_name, head_idx)]
            q_patch = q_patch.clone().to(q_clean.dtype).to(q_clean.device)
            if q_patch.dim() == 2 and q_clean.dim() == 3:
                q_patch = q_patch.unsqueeze(1)  # Now [batch, 1, head_dim]
            q_patch_proj = q_patch @ projection
            q_clean_proj = q_clean @ projection
            repr[:, head_idx, token_indices, :] += (q_patch_proj - q_clean_proj)

        repr = repr.transpose(1, 2).view(
            batch_size, seq_len, n_heads * head_dim
        )
        return repr

    with baukit.TraceDict(
        module=mt._model, layers=q_proj_modules, edit_output=perform_patch
    ):
        output = mt._model(**destination_tokenized)

    return output


def get_optimal_component_mask(
    mt: ModelandTokenizer,
    train_set: list[tuple[SelectionSample, SelectionSample]],
    q_proj_basis_directions: dict[tuple[int, int], torch.Tensor],
    learning_rate: float = 1e-3,
    n_epochs: int = 5,
    lamb: float = 1e-3,
    batch_size: int = 4,
    query_indices: int = [-1],
    save_path: PathLike | None = None,
    save_step: int = 5,
    loss_fn: Literal["promote_suppress", "match_gold"] = "match_gold",
):
    hparams = {
        "learning_rate": learning_rate,
        "n_epochs": n_epochs,
        "lamb": lamb,
        "batch_size": batch_size,
        "loss_fn": loss_fn,
    }
    loss_fn = {
        "promote_suppress": promote_target_suppress_distractors,
        "match_gold": match_gold_logit_distribution,
    }[loss_fn]
    logger.debug(f"Training with hparams: {hparams}")
    # n_layer = mt.n_layer
    # n_heads = mt.config.num_attention_heads
    head_dim = get_module_nnsight(
        mt._model, mt.attn_module_name_format.format(0)
    ).head_dim

    masks = {}
    for layer_idx, head_idx in q_proj_basis_directions.keys():
        masks[(layer_idx, head_idx)] = torch.ones(
            (head_dim,), dtype=mt.dtype, requires_grad=True, device=mt.device
        )

    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

    optimizer = AdamW([mask for mask in masks.values()], lr=learning_rate)
    losses = []

    all_q_proj_modules = []
    query_locations = []
    all_heads = list(q_proj_basis_directions.keys())
    for layer_idx, head_idx in all_heads:
        module_name = mt.attn_module_name_format.format(layer_idx) + ".q_proj"
        all_q_proj_modules.append(module_name)
        query_locations.extend(
            (layer_idx, head_idx, query_idx)
            for query_idx in query_indices
        )

    batches = []
    for batch_start in range(0, len(train_set), batch_size):
        batches.append(train_set[batch_start : batch_start + batch_size])

    def build_projections(masks):
        projections = {}
        for layer_idx, head_idx in all_heads:
            basis_directions = q_proj_basis_directions[(layer_idx, head_idx)]
            mask = masks[(layer_idx, head_idx)].to(basis_directions.dtype).to(basis_directions.device)
            masked_basis = basis_directions * mask[:, None]
            projections[(layer_idx, head_idx)] = masked_basis.T @ masked_basis
        return projections

    @torch.no_grad()
    def save_projections(save_file: PathLike):
        os.makedirs(os.path.dirname(save_file), exist_ok=True)
        optimal_masks = {key: mask.clone().round() for key, mask in masks.items()}
        with torch.no_grad():
            final_projections = build_projections(optimal_masks)
        torch.save(
            {"projections": final_projections, "masks": optimal_masks, "hparams": hparams},
            save_file,
        )
        del optimal_masks, final_projections
        free_gpu_cache()
        return

    logger.info("Starting training...")

    for epoch in range(n_epochs):
        epoch_loss = 0
        for batch_idx, batch in enumerate(batches):
            optimizer.zero_grad()

            batch_size_actual = len(batch)

            clean_samples, patch_samples = zip(*batch)
            prompts = []
            prompts.extend([sample.prompt() for sample in clean_samples])
            prompts.extend([sample.prompt() for sample in patch_samples])
            tokenized = prepare_input(prompts=prompts, tokenizer=mt)
            clean_tokenized = TokenizerOutput(
                data={k: v[: len(clean_samples), :] for k, v in tokenized.items()}
            )
            patch_tokenized = TokenizerOutput(
                data={k: v[len(clean_samples) :, :] for k, v in tokenized.items()}
            )

            projections = build_projections(masks=masks)

            output = apply_q_proj_patch_with_projection(
                mt=mt,
                source_tokenized=patch_tokenized,
                destination_tokenized=clean_tokenized,
                projections=projections,
                token_indices=query_indices,
            )

            logits = output.logits[:, -1, :]

            target_loss, loss_dict = loss_fn(
                mt=mt,
                source_samples=patch_samples,
                destination_samples=clean_samples,
                patched_logits=logits,
            )

            # mask loss
            mask_l1_loss = None
            for mask in masks.values():
                mask = mask.float()
                if mask_l1_loss is None:
                    mask_l1_loss = lamb * mask.norm(p=1)
                else:
                    mask_l1_loss += lamb * mask.norm(p=1).to(mask_l1_loss.device)

            loss = target_loss.float() + mask_l1_loss
            # loss = mask_l1_loss
            loss_dict_indv = (
                f"{', '.join([f'{k}={v:.3f}' for k, v in loss_dict.items()])}"
            )
            logger.debug(
                f"Epoch={epoch+1} | {batch_idx=} |>> {target_loss.item():.4f} [{loss_dict_indv}] + {mask_l1_loss.item():.4f} = {loss.item():.4f}"
            )

            loss.backward()
            # checking if gradients are flowing
            # for key, mask in list(masks.items())[:5]:
            #     if mask.grad is not None:
            #         print(f"{key}: grad norm = {mask.grad.norm().item():.6f}")
            #     else:
            #         print(f"{key}: NO GRADIENT!")
            optimizer.step()

            with torch.no_grad():
                for mask in masks.values():
                    mask.clamp_(0, 1)
                    # mask += 1e-4  # to avoid zero gradients

            # print(f"Mask sample values: {list(masks.values())[0][:5]}")  # First 5 elements
            # print(f"Mask mean: {list(masks.values())[0].mean().item()}")

            epoch_loss += loss.item() * batch_size_actual
            losses.append(loss.item())

        epoch_loss /= len(train_set)
        logger.info(f"Epoch {epoch+1}/{n_epochs} completed. Avg Loss: {epoch_loss:.4f}")

        mt._model.zero_grad()
        del (
            projections,
            output,
            logits,
        )
        free_gpu_cache()

        if save_path is not None and (
            (epoch + 1) % save_step == 0 or (epoch + 1) == n_epochs
        ):
            weight_path = os.path.join(save_path, f"epoch_{epoch+1}.pt")
            os.makedirs(os.path.dirname(weight_path), exist_ok=True)
            save_projections(save_file=weight_path)
            logger.info(f"Saved optimal projections to {weight_path}")

    final_masks = {key: mask.detach().round() for key, mask in masks.items()}
    final_projections = build_projections(final_masks)
    free_gpu_cache()
    return final_projections, final_masks, losses

In [ ]:
mt._model.zero_grad()
free_gpu_cache()

optimized_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "selection/test_svd_proj",
    mt.name.split("/")[-1],
    f"{TASK_CLS.task_name}",
)

projections, masks, losses = get_optimal_component_mask(
    mt=mt,
    train_set=train_set,
    q_proj_basis_directions=q_proj_basis_directions,
    query_indices=[-1],
    save_path=optimized_path,
    loss_fn="match_gold",
    learning_rate=1e-2,
    n_epochs=5,
    lamb=1e-4,
    batch_size=32,
    save_step=5,
)

2025-10-07 17:33:49 __main__ DEBUG    Training with hparams: {'learning_rate': 0.01, 'n_epochs': 2, 'lamb': 0.0002, 'batch_size': 16, 'loss_fn': 'match_gold'}
2025-10-07 17:33:49 __main__ INFO     Starting training...
2025-10-07 17:33:55 __main__ DEBUG    Epoch=1 | batch_idx=0 |>> 2.4844 [kldiv_loss=2.484] + 131.0650 = 133.5494
2025-10-07 17:34:11 __main__ DEBUG    Epoch=1 | batch_idx=1 |>> 2.4062 [kldiv_loss=2.406] + 129.6281 = 132.0344
2025-10-07 17:34:21 __main__ DEBUG    Epoch=1 | batch_idx=2 |>> 1.9219 [kldiv_loss=1.922] + 128.2799 = 130.2018
2025-10-07 17:34:31 __main__ DEBUG    Epoch=1 | batch_idx=3 |>> 1.4922 [kldiv_loss=1.492] + 126.9485 = 128.4407
2025-10-07 17:34:41 __main__ DEBUG    Epoch=1 | batch_idx=4 |>> 1.5234 [kldiv_loss=1.523] + 125.6213 = 127.1448
2025-10-07 17:34:50 __main__ DEBUG    Epoch=1 | batch_idx=5 |>> 2.5781 [kldiv_loss=2.578] + 124.2969 = 126.8750
2025-10-07 17:34:58 __main__ DEBUG    Epoch=1 | batch_idx=6 |>> 1.1328 [kldiv_loss=1.133] + 122.9793 = 124.112

In [28]:
loaded_opt_results = torch.load(os.path.join(optimized_path, "epoch_2.pt"), weights_only=False)

In [29]:
loaded_opt_results.keys()

dict_keys(['projections', 'masks', 'hparams'])